<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/p020_Les_Tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [2]:
pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 6.8 MB/s 


In [3]:
pip install -q tensorflow_datasets

In [17]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

# Overview

3 grands tokenizers présents dans `tensorflow_text` :
 * `text.BertTokenizer` : high level function qui prend en input des sentences et un vocabulaire
 * `text.WordpieceTokenizer` : il s'agit de l'implémentation de l'algorithme WordPiece algorithm, nécessite pas mal d'étapes avant de l'appliquer
 * `text.SentencepieceTokenizer` :  il s'agit de l'api la plus complexe voir https://www.tensorflow.org/text/api_docs/python/text/SentencepieceTokenizer pour plus d'informations. Il semblerait que ce soit à l'aide de ce tokenizer qu'on puisse tokeniser des langues comme le chinois ou l'hébreu

# Objectif
L'objectif est de construire un vocabulaire de style WordPiece en partant des mots puis en allant vers les subwords

# Data
Il s'agit de phrases rédigées à la fois en anglais et en portugais qu'on utilise pour les tâches de traduction automatique

In [18]:
# Définition de pwd
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()
print(pwd)

/content


In [8]:
# Téléchargement des données via la librairie tfds
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incomplete5X6RT3/ted_hrlr_translate-train.tfrecord


...

  0%|          | 0/1193 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incomplete5X6RT3/ted_hrlr_translate-test.tfrecord


  0%|          | 0/1803 [00:00<?, ? examples/s]

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Pour décoder utiliser la méthode .decode() d'un objet string
for pt, en in train_examples.take(1):
  print("Portuguese: ", pt.numpy().decode('utf-8'))
  print("English:   ", en.numpy().decode('utf-8'))

In [11]:
# Dataset splitting
train_en = train_examples.map(lambda pt, en: en)
train_pt = train_examples.map(lambda pt, en: pt)

# Génération du vocabulaire
 * Afin de calculer le vocabulaire d'un corpus, on utilise la fonction `bert_vocab.bert_vocab_from_dataset()` qui prend les inputs suivants : 
   * Un dataset Tensorflow
   * Un dictionnaire d'arguments `bert_vocab_args`:
     * `reserved_tokens` : comprend les tokens spéciaux
     * `vocab_size` : la taille du vocabulaire, ce qui va contraindre le tokenizer à ne sélectionner que les 8000 mots les plus fréquents
 * Cela produit une liste python classique en sortie
 * La bonne pratique est d'enregistrer ce vocabulaire dans un fichier texte
   

In [19]:
# Définition du dictionnaire d'arguments pour la création du vocabulaire bert_vocab.bert_vocab_from_dataset()

bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [20]:
# Création du vocabulaire
# Ca crée une liste python

pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_pt.batch(1000).prefetch(2),
    **bert_vocab_args
)

print(len(pt_vocab))

In [23]:
# Enregistrement du vocabulaire sur un fichier texte
# Définition de la fonction d'enregistrement

def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

# Enregistrement du vocabulaire
write_vocab_file('pt_vocab.txt', pt_vocab)

# Construction du tokenizer
 * Avec `text.BertTokenizer` prend comme inputs 
   * Le fichier text contenant le vocabulaire
   * Un dictionnaire de parameters qui peut comprendre beaucoup d'arguments mais dans notre cas, on utilisera juste un argument

In [25]:
# Print du dictionnaire d'arguments qu'on utilisera dans la création du tokenizer
print(bert_tokenizer_params)

# Création du tokenizer portugais
pt_tokenizer = text.BertTokenizer('pt_vocab.txt', **bert_tokenizer_params)

{'lower_case': True}


# Encodage du texte
Maintenant qu'on a notre tokenizer on peut l'utiliser pour encoder notre texte en portugais
 * Afin de tokenizer, appliquez simplement le tokenizer sur un batch de sentences comme `pt_tokenizer.tokenize(portougech_batch)` 
 * Afin de detokenizer un tensor, utilisez  `tf.gather(le_vocab, batch_ragged)`
 * La version plus propre de detokenization `my_tokenizer.detokenize(batch)`
 * Attention l'output d'une detokenization n'est pas strictement égale à l'input d'une tokenization. Certains mots sont lowerisés, la ponctuation est le plus souvent removed et d'autres mots sont splitted

In [37]:
for portougech_batch in train_pt.batch(3).take(1):
  for e in portougech_batch:
    print(e.numpy().decode("utf-8"))
    print('')

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .

mas e se estes fatores fossem ativos ?

mas eles não tinham a curiosidade de me testar .



In [38]:
# Tokenize the examples -> (batch, word, word-piece)
# Merge the word and word-piece axes -> (batch, tokens)

token_batch = pt_tokenizer.tokenize(portougech_batch)
token_batch = token_batch.merge_dims(-2,-1)
print(token_batch)

<tf.RaggedTensor [[44, 115, 6402, 148, 40, 887, 14, 3936, 40, 463, 2715, 94, 2047, 14, 84,
  44, 40, 117, 1328, 2721, 818, 539, 16]                                 ,
 [99, 44, 89, 199, 2836, 1336, 3996, 32],
 [99, 131, 88, 383, 40, 2673, 83, 110, 1972, 16]]>


In [46]:
# Detokenize batch de ragged tensor
txt_tokens = tf.gather(pt_vocab, token_batch)
print(txt_tokens)
print('')

# Original
print(portougech_batch)
print('')

# Merge text tokens
txt_tokens = tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)
print(txt_tokens.numpy())

tf.Tensor(
[b'e' b'quando' b'melhora' b'##mos' b'a' b'procura' b',' b'tiramos' b'a'
 b'unica' b'vantagem' b'da' b'impressao' b',' b'que' b'e' b'a' b'ser'
 b'##en' b'##di' b'##p' b'##idade' b'.' b'mas' b'e' b'se' b'estes'
 b'fatores' b'fossem' b'ativos' b'?' b'mas' b'eles' b'nao' b'tinham' b'a'
 b'curiosidade' b'de' b'me' b'testar' b'.'], shape=(41,), dtype=string)

tf.Tensor(
[b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .'
 b'mas e se estes fatores fossem ativos ?'
 b'mas eles n\xc3\xa3o tinham a curiosidade de me testar .'], shape=(3,), dtype=string)

b'e quando melhora ##mos a procura , tiramos a unica vantagem da impressao , que e a ser ##en ##di ##p ##idade . mas e se estes fatores fossem ativos ? mas eles nao tinham a curiosidade de me testar .'


In [51]:
print('original batch')
print(portougech_batch)
print('')

token_batch = pt_tokenizer.tokenize(portougech_batch)
print('tokenized batch')
print(token_batch)
print('')

detonkeinze_batch = pt_tokenizer.detokenize(token_batch)
print('detokenized batch')
print(detonkeinze_batch)
print('')

original batch
tf.Tensor(
[b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .'
 b'mas e se estes fatores fossem ativos ?'
 b'mas eles n\xc3\xa3o tinham a curiosidade de me testar .'], shape=(3,), dtype=string)

tokenized batch
<tf.RaggedTensor [[[44], [115], [6402, 148], [40], [887], [14], [3936], [40], [463], [2715],
  [94], [2047], [14], [84], [44], [40], [117, 1328, 2721, 818, 539], [16]] ,
 [[99], [44], [89], [199], [2836], [1336], [3996], [32]],
 [[99], [131], [88], [383], [40], [2673], [83], [110], [1972], [16]]]>

detokenized batch
<tf.RaggedTensor [[[b'e'],
  [b'quando'],
  [b'melhoramos'],
  [b'a'],
  [b'procura'],
  [b','],
  [b'tiramos'],
  [b'a'],
  [b'unica'],
  [b'vantagem'],
  [b'da'],
  [b'impressao'],
  [b','],
  [b'que'],
  [b'e'],
  [b'a'],
  [b'serendipidade'],
  [b'.']]            , [[b'mas'],
                        [b'e'],
                        [b'se'],
                        [b'estes'],
     

# Custom tokenization et detokenization
 * **Tokenization** : Il est d'usage dans la plupart des uses cases de rajouter un token START et END au début et à la fin de chaque phrase. La bonne pratique veut donc qu'on crée une fonction qui va rajouter ces tokens aux sentences déjà tokenizées
 * **Detokenization** : Il est d'usage de créer une fonction de détokenization qui prend en input un batch de ragged tensors de strings et qui va remove les reserved tokens excepté les < UNK > tokens. Il va ensuite merger les text tokens

In [54]:
# Récupération des tokens id pour le token start et end
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")
print(START)
print(END)

#Fonction chargé de rajouter ces tokens
def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

# Les tokens 2 et 3 sont rajoutés en début et chaque fin de phrase
token_batch = pt_tokenizer.tokenize(portougech_batch)
token_batch = token_batch.merge_dims(-2,-1)
add_start_end(token_batch)

tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)


In [59]:
# Fonction qui remove les reserved tokens d'un batch de ragged token text
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

print('les tokens spéciaux')
print(reserved_tokens)
print('')

token_batch = pt_tokenizer.tokenize(portougech_batch)
token_batch = token_batch.merge_dims(-2,-1)
print('Le token batch a detokenizé, il a été merge au préalable')
print(token_batch)
print('')

detoken_batch = pt_tokenizer.detokenize(token_batch)
print('Le batch de text token mais pas encore cleaned up')
print(detoken_batch)
print('')

detoken_batch_cleaned = cleanup_text(reserved_tokens,detoken_batch)
print('le batch de tokens cleaned up')
print(detoken_batch_cleaned)

les tokens spéciaux
['[PAD]', '[UNK]', '[START]', '[END]']

Le token batch a detokenizé, il a été merge au préalable
<tf.RaggedTensor [[44, 115, 6402, 148, 40, 887, 14, 3936, 40, 463, 2715, 94, 2047, 14, 84,
  44, 40, 117, 1328, 2721, 818, 539, 16]                                 ,
 [99, 44, 89, 199, 2836, 1336, 3996, 32],
 [99, 131, 88, 383, 40, 2673, 83, 110, 1972, 16]]>

Le batch de text token mais pas encore cleaned up
<tf.RaggedTensor [[b'e', b'quando', b'melhoramos', b'a', b'procura', b',', b'tiramos', b'a',
  b'unica', b'vantagem', b'da', b'impressao', b',', b'que', b'e', b'a',
  b'serendipidade', b'.']                                                  ,
 [b'mas', b'e', b'se', b'estes', b'fatores', b'fossem', b'ativos', b'?'],
 [b'mas', b'eles', b'nao', b'tinham', b'a', b'curiosidade', b'de', b'me',
  b'testar', b'.']                                                       ]>

le batch de tokens cleaned up
tf.Tensor(
[b'e quando melhoramos a procura , tiramos a unica vantagem da im

# Export Custom Tokenizer
Il se trouve qu'en subclassant `tf.Module` on peut exporter un Custom Tokenizer. Voici les étapes avec un peu plus de précisions :    
 * Subclassing de tf.Module
 * Instanciation d'un module
 * Définition d'un attribut 'tokenizer english'
 * Définition d'un attribut 'tokenizer portugese'
 * Sauvegarde du module via `tf.saved_model.save()`
   * Le module en question
   * une string qui représente le nom du tokenizer
 * Pour loader le module tokenizer :    
   * `reloaded_tokenizers = tf.saved_model.load(model_name)`
   * Accès au tokenizer portugais : `reloaded_tokenizers.pt`
 * La bonne pratique est de compressé le fichier représentant le custom tokenizer class avec : `zip -r {model_name}.zip {model_name}`

In [60]:
# Le Subclassing
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:   

    # Include a tokenize signature for a batch of strings. 
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

In [ ]:
# Export du tokenizer
tokenizers = tf.Module()
tokenizers.pt = CustomTokenizer(reserved_tokens, 'pt_vocab.txt')
tokenizers.en = CustomTokenizer(reserved_tokens, 'en_vocab.txt')
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.saved_model.save(tokenizers, model_name)

In [ ]:
# Load Tokenizer
reloaded_tokenizers = tf.saved_model.load(model_name)
tokens = reloaded_tokenizers.en.tokenize(['Hello TensorFlow!'])

In [ ]:
# * * * TIPS * * * 

# pour décoder une string qui est au format bytes : ma_string.decode('utf-8')
# Accès à un maximum de dataset avec la librairie TFDS
# installation de tensorflow text : pip install -q -U "tensorflow-text==2.8.*"
# installation de tensorflow dataset : pip install -q tensorflow_datasets

# Pour écrire une liste python sur un fichier :
#  with open(filepath, 'w') as f:
#    for token in vocab:
#      print(token, file=f)

# Pour merger un ragged tensor, utilisez la methode .merge_dims()
# token_batch.merge_dims(-2,-1)

# Pour merger du texte tokens
# tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)